In [ ]:
import os
from glob import glob
import pickle
import itertools
import numpy as np
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

### Graph imports ###
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd

### Audio import ###
import librosa
import IPython
from IPython.display import Audio

### Plot imports ###
from IPython.display import Image
import matplotlib.pyplot as plt

### Time Distributed ConvNet imports ###
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, TimeDistributed, concatenate
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, BatchNormalization, LeakyReLU, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K
from keras.utils import np_utils
from keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder

### Warning ###
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [ ]:
# Audio file path and names
RAV = '/content/drive/My Drive/audio_speech_actors_01-24/'


In [ ]:
TESS = "/content/drive/My Drive/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/"

SAVEEJE = "/content/drive/My Drive/AudioData/AudioData/JE/"
SAVEEDC = "/content/drive/My Drive/AudioData/AudioData/DC/"
SAVEEJK = "/content/drive/My Drive/AudioData/AudioData/JK/"
SAVEEKL = "/content/drive/My Drive/AudioData/AudioData/KL/"
CREMA = "/content/drive/My Drive/AudioWAV/"



In [ ]:
dic  =  {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
dic  =  {1:'male_neutral', 2:'male_calm', 3:'male_happy', 4:'male_sad', 5:'male_angry', 6:'male_fear', 7:'male_disgust', 8:'male_surprise',9:'female_neutral', 10:'female_calm', 11:'female_happy', 12:'female_sad', 13:'female_angry', 14:'female_fear', 15:'female_disgust', 16:'female_surprise'}

In [ ]:
# dir_list = os.listdir(RAV)
# dir_list.sort()

# emotion = []
# path = []
# for i in dir_list:
#     fname = os.listdir(RAV + i)
#     for f in fname:
#         part = f.split('.')[0].split('-')
#         emotion.append(int(part[2]))
#         path.append(RAV + i + '/' + f)

# RAV_df = pd.DataFrame() 
# RAV_df = pd.concat([RAV_df,pd.DataFrame(path, columns = ['path'])],axis=1)

In [ ]:
dir_list = os.listdir(RAV)
dir_list.sort()

emotion = []
gender = []
path = []
for i in dir_list:
    fname = os.listdir(RAV + i)
    for f in fname:
        part = f.split('.')[0].split('-')
        emotion.append(int(part[2]))
        temp = int(part[6])
        if temp%2 == 0:
            temp = "female"
        else:
            temp = "male"
        gender.append(temp)
        path.append(RAV + i + '/' + f)

        
RAV_df = pd.DataFrame(emotion)
RAV_df = RAV_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
RAV_df = pd.concat([pd.DataFrame(gender),RAV_df],axis=1)
RAV_df.columns = ['gender','emotion']
RAV_df['labels'] =RAV_df.gender + '_' + RAV_df.emotion
RAV_df['source'] = 'RAVDESS'  
RAV_df = pd.concat([RAV_df,pd.DataFrame(path, columns = ['path'])],axis=1)
RAV_df = RAV_df.drop(['gender', 'emotion'], axis=1)
RAV_df.labels.value_counts()

FileNotFoundError: ignored

In [ ]:
# Run one example 
dir_list = os.listdir(SAVEEJE)
dir_list[0:5]
# parse the filename to get the emotions
emotion=[]
path = []
for i in dir_list:
    if i[-8:-6]=='a':
        emotion.append('male_angry')
    elif i[-8:-6]=='d':
        emotion.append('male_disgust')
    elif i[-8:-6]=='f':
        emotion.append('male_fear')
    elif i[-8:-6]=='h':
        emotion.append('male_happy')
    elif i[-8:-6]=='n':
        emotion.append('male_neutral')
    elif i[-8:-6]=='sa':
        emotion.append('male_sad')
    elif i[-8:-6]=='su':
        emotion.append('male_surprise')
    else:
        emotion.append('male_error') 
    path.append(SAVEEJE + i)
    
# Now check out the label count distribution 
SAVEE_dfje = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_dfje['source'] = 'SAVEEJE'
SAVEE_dfje = pd.concat([SAVEE_dfje, pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_dfje.labels.value_counts()

In [ ]:
# parse the filename to get the emotions
dir_list = os.listdir(SAVEEDC)
emotion=[]
path = []
for i in dir_list:
    if i[-8:-6]=='a':
        emotion.append('male_angry')
    elif i[-8:-6]=='d':
        emotion.append('male_disgust')
    elif i[-8:-6]=='f':
        emotion.append('male_fear')
    elif i[-8:-6]=='h':
        emotion.append('male_happy')
    elif i[-8:-6]=='n':
        emotion.append('male_neutral')
    elif i[-8:-6]=='sa':
        emotion.append('male_sad')
    elif i[-8:-6]=='su':
        emotion.append('male_surprise')
    else:
        emotion.append('male_error') 
    path.append(SAVEEDC + i)
    
# Now check out the label count distribution 
SAVEE_dfdc = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_dfdc['source'] = 'SAVEEDC'
SAVEE_dfdc = pd.concat([SAVEE_dfdc, pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_dfdc.labels.value_counts()

In [ ]:
# parse the filename to get the emotions
dir_list = os.listdir(SAVEEJK)
emotion=[]
path = []
for i in dir_list:
    if i[-8:-6]=='a':
        emotion.append('male_angry')
    elif i[-8:-6]=='d':
        emotion.append('male_disgust')
    elif i[-8:-6]=='f':
        emotion.append('male_fear')
    elif i[-8:-6]=='h':
        emotion.append('male_happy')
    elif i[-8:-6]=='n':
        emotion.append('male_neutral')
    elif i[-8:-6]=='sa':
        emotion.append('male_sad')
    elif i[-8:-6]=='su':
        emotion.append('male_surprise')
    else:
        emotion.append('male_error') 
    path.append(SAVEEJK + i)
    
# Now check out the label count distribution 
SAVEE_dfjk = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_dfjk['source'] = 'SAVEEJK'
SAVEE_dfjk = pd.concat([SAVEE_dfjk, pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_dfjk.labels.value_counts()

In [ ]:
# parse the filename to get the emotions
dir_list = os.listdir(SAVEEKL)
emotion=[]
path = []
for i in dir_list:
    if i[-8:-6]=='a':
        emotion.append('male_angry')
    elif i[-8:-6]=='d':
        emotion.append('male_disgust')
    elif i[-8:-6]=='f':
        emotion.append('male_fear')
    elif i[-8:-6]=='h':
        emotion.append('male_happy')
    elif i[-8:-6]=='n':
        emotion.append('male_neutral')
    elif i[-8:-6]=='sa':
        emotion.append('male_sad')
    elif i[-8:-6]=='su':
        emotion.append('male_surprise')
    else:
        emotion.append('male_error') 
    path.append(SAVEEKL + i)
    
# Now check out the label count distribution 
SAVEE_dfkl = pd.DataFrame(emotion, columns = ['labels'])
SAVEE_dfkl['source'] = 'SAVEEKL'
SAVEE_dfkl = pd.concat([SAVEE_dfkl, pd.DataFrame(path, columns = ['path'])], axis = 1)
SAVEE_dfkl.labels.value_counts()

In [ ]:
dir_list = os.listdir(TESS)
dir_list.sort()
dir_list
path = []
emotion = []

for i in dir_list:
    fname = os.listdir(TESS + i)
    for f in fname:
        if i == 'OAF_angry' or i == 'YAF_angry':
            emotion.append('female_angry')
        elif i == 'OAF_disgust' or i == 'YAF_disgust':
            emotion.append('female_disgust')
        elif i == 'OAF_Fear' or i == 'YAF_fear':
            emotion.append('female_fear')
        elif i == 'OAF_happy' or i == 'YAF_happy':
            emotion.append('female_happy')
        elif i == 'OAF_neutral' or i == 'YAF_neutral':
            emotion.append('female_neutral')                                
        elif i == 'OAF_Pleasant_surprise' or i == 'YAF_pleasant_surprised':
            emotion.append('female_surprise')               
        elif i == 'OAF_Sad' or i == 'YAF_sad':
            emotion.append('female_sad')
        else:
            emotion.append('Unknown')
        path.append(TESS + i + "/" + f)

TESS_df = pd.DataFrame(emotion, columns = ['labels'])
TESS_df['source'] = 'TESS'
TESS_df = pd.concat([TESS_df,pd.DataFrame(path, columns = ['path'])],axis=1)
TESS_df.labels.value_counts()

In [ ]:
dir_list = os.listdir(CREMA)
gender = []
emotion = []
path = []
female = [1002,1003,1004,1006,1007,1008,1009,1010,1012,1013,1018,1020,1021,1024,1025,1028,1029,1030,1037,1043,1046,1047,1049,
          1052,1053,1054,1055,1056,1058,1060,1061,1063,1072,1073,1074,1075,1076,1078,1079,1082,1084,1089,1091]

for i in dir_list: 
    part = i.split('_')
    if int(part[0]) in female:
        temp = 'female'
    else:
        temp = 'male'
    gender.append(temp)
    if part[2] == 'SAD' and temp == 'male':
        emotion.append('male_sad')
    elif part[2] == 'ANG' and temp == 'male':
        emotion.append('male_angry')
    elif part[2] == 'DIS' and temp == 'male':
        emotion.append('male_disgust')
    elif part[2] == 'FEA' and temp == 'male':
        emotion.append('male_fear')
    elif part[2] == 'HAP' and temp == 'male':
        emotion.append('male_happy')
    elif part[2] == 'NEU' and temp == 'male':
        emotion.append('male_neutral')
    elif part[2] == 'SAD' and temp == 'female':
        emotion.append('female_sad')
    elif part[2] == 'ANG' and temp == 'female':
        emotion.append('female_angry')
    elif part[2] == 'DIS' and temp == 'female':
        emotion.append('female_disgust')
    elif part[2] == 'FEA' and temp == 'female':
        emotion.append('female_fear')
    elif part[2] == 'HAP' and temp == 'female':
        emotion.append('female_happy')
    elif part[2] == 'NEU' and temp == 'female':
        emotion.append('female_neutral')
    else:
        emotion.append('Unknown')
    path.append(CREMA + i)
    
CREMA_df = pd.DataFrame(emotion, columns = ['labels'])
CREMA_df['source'] = 'CREMA'
CREMA_df = pd.concat([CREMA_df,pd.DataFrame(path, columns = ['path'])],axis=1)
CREMA_df.labels.value_counts()


In [ ]:
RAV_df  = pd.DataFrame()
RAV_df = pd.concat([SAVEE_dfje,SAVEE_dfdc,SAVEE_dfjk,SAVEE_dfkl, RAV_df, TESS_df, CREMA_df],axis=0)

In [ ]:
# dir_list = os.listdir(RAV)
# dir_list.sort()

# emotion = []
# path = []
# for i in dir_list:
#     fname = os.listdir(RAV + i)
#     for f in fname:
#         part = f.split('.')[0].split('-')
#         emotion.append(int(part[2]))
#         path.append(RAV + i + '/' + f)

# RAV_df = pd.DataFrame() 
# RAV_df = pd.concat([RAV_df,pd.DataFrame(path, columns = ['path'])],axis=1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
RAV_df.head()

In [ ]:
RAV_df.path

In [ ]:
signal = []

# Sample rate (16.0 kHz)
sample_rate = 16000     

# Max pad lenght (3.0 sec)
max_pad_len = 49100

for index,path in enumerate(RAV_df.path):
    X, sample_rate = librosa.load(path
                                  ,duration=3
                                  ,offset=0.5
                                 )
    sample_rate = np.array(sample_rate)
    
    y = zscore(X)
        
    # Padding or truncated signal 
    if len(y) < max_pad_len:    
        y_padded = np.zeros(max_pad_len)
        y_padded[:len(y)] = y
        y = y_padded
    elif len(y) > max_pad_len:
        y = np.asarray(y[:max_pad_len])

    # Add to signal list
    signal.append(y)
    

In [ ]:
labels = np.asarray(emotion).ravel()

In [ ]:
labels.shape

In [ ]:
np.unique(labels)

In [ ]:
def noisy_signal(signal, snr_low=15, snr_high=30, nb_augmented=2):
    
    # Signal length
    signal_len = len(signal)

    # Generate White noise
    noise = np.random.normal(size=(nb_augmented, signal_len))
    
    # Compute signal and noise power
    s_power = np.sum((signal / (2.0 ** 15)) ** 2) / signal_len
    n_power = np.sum((noise / (2.0 ** 15)) ** 2, axis=1) / signal_len
    
    # Random SNR: Uniform [15, 30]
    snr = np.random.randint(snr_low, snr_high)
    
    # Compute K coeff for each noise
    K = np.sqrt((s_power / n_power) * 10 ** (- snr / 10))
    K = np.ones((signal_len, nb_augmented)) * K
    
    # Generate noisy signal
    return signal + K.T * noise

In [ ]:
print("Data Augmentation: START")
augmented_signal = list(map(noisy_signal, signal))
print("Data Augmentation: END!")

In [ ]:
def mel_spectrogram(y, sr=16000, n_fft=512, win_length=256, hop_length=128, window='hamming', n_mels=128, fmax=4000):
    
    # Compute spectogram
    mel_spect = np.abs(librosa.stft(y, n_fft=n_fft, window=window, win_length=win_length, hop_length=hop_length)) ** 2
    
    # Compute mel spectrogram
    mel_spect = librosa.feature.melspectrogram(S=mel_spect, sr=sr, n_mels=n_mels, fmax=fmax)
    
    # Compute log-mel spectrogram
    mel_spect = librosa.power_to_db(mel_spect, ref=np.max)
    
    return mel_spect

In [ ]:
mel_spect = np.asarray(list(map(mel_spectrogram, signal)))
augmented_mel_spect = [np.asarray(list(map(mel_spectrogram, augmented_signal[i]))) for i in range(len(augmented_signal))]

In [ ]:
MEL_SPECT_train, MEL_SPECT_test, AUG_MEL_SPECT_train, AUG_MEL_SPECT_test, label_train, label_test = train_test_split(mel_spect, augmented_mel_spect,labels, test_size=0.2,random_state=1)

# Build augmented labels and train
aug_label_train = np.asarray(list(itertools.chain.from_iterable([[label] * 2 for label in label_train])))
AUG_MEL_SPECT_train = np.asarray(list(itertools.chain.from_iterable(AUG_MEL_SPECT_train)))

# Concatenate original and augmented
X_train = np.concatenate((MEL_SPECT_train, AUG_MEL_SPECT_train))
y_train = np.concatenate((label_train, aug_label_train))

# Build test set
X_test = MEL_SPECT_test
y_test = label_test

In [ ]:
X_train.shape

In [ ]:
y_train

In [ ]:
win_ts = 128
hop_ts = 64

# Split spectrogram into frames
def frame(x, win_step=128, win_size=64):
    nb_frames = 1 + int((x.shape[2] - win_size) / win_step)
    frames = np.zeros((x.shape[0], nb_frames, x.shape[1], win_size)).astype(np.float32)
    for t in range(nb_frames):
        frames[:,t,:,:] = np.copy(x[:,:,(t * win_step):(t * win_step + win_size)]).astype(np.float32)
    return frames

# Frame for TimeDistributed model
X_train = frame(X_train, hop_ts, win_ts)
X_test = frame(X_test, hop_ts, win_ts)

In [ ]:
lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(np.ravel(y_train)))
y_test = np_utils.to_categorical(lb.transform(np.ravel(y_test)))

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] , X_train.shape[2], X_train.shape[3], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1] , X_test.shape[2], X_test.shape[3], 1)

In [ ]:
K.clear_session()

# Define two sets of inputs: MFCC and FBANK
input_y = Input(shape=X_train.shape[1:], name='Input_MELSPECT')

## First LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_1_MELSPECT')(input_y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_1_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_1_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'), name='MaxPool_1_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_1_MELSPECT')(y)     

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_2_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_2_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_2_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_2_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_2_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_3_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_3_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_3_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_3_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_3_MELSPECT')(y)

## Second LFLB (local feature learning block)
y = TimeDistributed(Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same'), name='Conv_4_MELSPECT')(y)
y = TimeDistributed(BatchNormalization(), name='BatchNorm_4_MELSPECT')(y)
y = TimeDistributed(Activation('elu'), name='Activ_4_MELSPECT')(y)
y = TimeDistributed(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'), name='MaxPool_4_MELSPECT')(y)
y = TimeDistributed(Dropout(0.2), name='Drop_4_MELSPECT')(y)  

## Flat
y = TimeDistributed(Flatten(), name='Flat_MELSPECT')(y)                      
                               
# Apply 2 LSTM layer and one FC
y = LSTM(256, return_sequences=False, dropout=0.2, name='LSTM_1')(y)
y = Dense(y_train.shape[1], activation='softmax', name='FC')(y)

# Build final model
model = Model(inputs=input_y, outputs=y)


In [ ]:
model.compile(optimizer=SGD(lr=0.01, decay=1e-6, momentum=0.8), loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=30, verbose=1, mode='max')

# Fit model
history = model.fit(X_train, y_train, batch_size=84, epochs=200, validation_data=(X_test, y_test), callbacks=[early_stopping])

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
score

In [ ]:
model.save('[CNN-LSTM]M.h5')
model.save_weights('[CNN-LSTM]W.h5')

In [ ]:
model_name = 'Emotion_ModelLSTM.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Save model and weights at %s ' % model_path)

# Save the model to disk
model_json = model.to_json()
with open("model_json.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
preds = model.predict(X_test)

preds=preds.argmax(axis=1)
preds

In [ ]:
import sklearn.metrics as metrics


#y_pred_labels = np.argmax(preds, axis=1)  # only necessary if output has one-hot-encoding, shape=(n_samples)

confusion_matrix = metrics.confusion_matrix(y_true=y_test, y_pred=preds)  # shape=(12, 12)

In [ ]:
path_ = '/content/drive/My Drive/audio_speech_actors_01-24/Actor_01/03-01-07-02-01-01-01.wav'

In [ ]:
import IPython.display as ipd
ipd.Audio(path_)

In [ ]:
s = []

# Sample rate (16.0 kHz)
sample_rate = 16000     

# Max pad lenght (3.0 sec)
max_pad_len = 49100

X, sample_rate = librosa.load(path_,duration=3,offset=0.5)
sample_rate = np.array(sample_rate)

y = zscore(X)
    
# Padding or truncated signal 
if len(y) < max_pad_len:    
    y_padded = np.zeros(max_pad_len)
    y_padded[:len(y)] = y
    y = y_padded
elif len(y) > max_pad_len:
    y = np.asarray(y[:max_pad_len])

# Add to signal list
s.append(y)

In [ ]:
mel_spect = np.asarray(list(map(mel_spectrogram, s)))

In [ ]:
win_ts = 128
hop_ts = 64

# Split spectrogram into frames
def frame(x, win_step=128, win_size=64):
    nb_frames = 1 + int((x.shape[2] - win_size) / win_step)
    frames = np.zeros((x.shape[0], nb_frames, x.shape[1], win_size)).astype(np.float32)
    for t in range(nb_frames):
        frames[:,t,:,:] = np.copy(x[:,:,(t * win_step):(t * win_step + win_size)]).astype(np.float32)
    return frames

# Frame for TimeDistributed model
x = frame(mel_spect, hop_ts, win_ts)

In [ ]:
x = x.reshape(x.shape[0], x.shape[1] , x.shape[2], x.shape[3], 1)

In [ ]:
preds = model.predict(x)

In [ ]:
preds=preds.argmax(axis=1)
preds